# Run ADBench 
- Here we provide a demo for testing AD algorithms on the datasets proposed in ADBench.
- Feel free to evaluate any customized algorithm in ADBench.
- For reproducing the complete experiment results in ADBench, please run the code in the run.py file.

# 调参

In [1]:
# -*- coding:utf-8 -*-
import numpy as np
from glob import glob
import pickle
import random
from tqdm import tqdm
import pandas as pd
import argparse
from sklearn.utils import shuffle
#import sklearn.metrics
#import sklearn.preprocessing
import torch
#import torch.nn as nn
import torch.nn.functional as F
import delu
import rtdl
import scipy
import platform
from matplotlib import pyplot as plt
from myutils import Utils
import optuna
utils = Utils() # utils function



def  load_data(pkl_list,label=True):
    '''
    输入pkl的列表，进行文件加载
    label=True用来加载训练集
    label=False用来加载真正的测试集，真正的测试集无标签
    '''
    X = []
    y = []
    

    for  each_pkl in pkl_list:
        pic = open(each_pkl,'rb')
        item= pickle.load(pic)#下载pkl文件
        # 此处选取的是每个滑窗的最后一条数据，仅供参考，可以选择其他的方法，比如均值或者其他处理时序数据的网络
        # 此处选取了前7个特征，可以需求选取特征数量
        feature = item[0][:,0:7][-1]
        #feature = item[0][:,0:7][-1]
        #feature = item[0][:,0:7].mean(axis=0)
        #feature = np.append(item[0][:,0:7][-1],(item[0][:,3][-1] - item[0][:,4][-1])) #加max_single_volt - min_single_volt 一列为特征
        feature=np.append(feature,item[1]["mileage"])
        X.append(feature)
        if label:
            y.append(int(item[1]['label'][0]))
    X = np.vstack(X)
    if label:
        y = np.vstack(y)
    return X, y
    
def normalization(data): 
    """
    归一化数据
    """
    _mean = np.mean(data, axis=0)
    _std = np.std(data, axis=0)
    data = (data - _mean) / (_std + 1e-4)
    return data


class FTTransformer():
    '''
    The original code: https://yura52.github.io/rtdl/stable/index.html
    The original paper: "Revisiting Deep Learning Models for Tabular Data", NIPS 2019
    '''
    def __init__(self, seed:int, model_name:str, n_epochs=100, batch_size=64):

        self.seed = seed
        self.model_name = model_name
        self.utils = Utils()

        # device
        if model_name == 'FTTransformer':
            self.device = self.utils.get_device(gpu_specific=True)
        else:
            self.device = self.utils.get_device(gpu_specific=False)

        # Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
        # zero.improve_reproducibility(seed=self.seed)
        delu.improve_reproducibility(base_seed=int(self.seed))

        # hyper-parameter
        self.n_epochs = n_epochs # default is 1000
        self.batch_size = batch_size # default is 256

    def apply_model(self, x_num, x_cat=None):
        if isinstance(self.model, rtdl.FTTransformer):
            return self.model(x_num, x_cat)
        elif isinstance(self.model, (rtdl.MLP, rtdl.ResNet)):
            assert x_cat is None
            return self.model(x_num)
        else:
            raise NotImplementedError(
                f'Looks like you are using a custom model: {type(self.model)}.'
                ' Then you have to implement this branch first.'
            )

    @torch.no_grad()
    def evaluate(self, X, y=None):
        self.model.eval()
        score = []
        # for batch in delu.iter_batches(X[part], 1024):
        for batch in delu.iter_batches(X, self.batch_size):
            score.append(self.apply_model(batch))
        score = torch.cat(score).squeeze(1).cpu().numpy()
        score = scipy.special.expit(score)

        # calculate the metric
        if y is not None:
            target = y.cpu().numpy()
            metric = self.utils.metric(y_true=target, y_score=score)
        else:
            metric = {'aucroc': None, 'aucpr': None}

        return score, metric['aucpr']

    def fit(self, X_train, y_train, ratio=None,X_test=None,y_test=None,params=None):
        # set seed
        self.utils.set_seed(self.seed)
       
        #X_train, X_test_val, y_train, y_test_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
        # training set is used as the validation set in the anomaly detection task
        X = {'train': torch.from_numpy(X_train).float().to(self.device),
             'val': torch.from_numpy(X_train).float().to(self.device)}

        y = {'train': torch.from_numpy(y_train).float().to(self.device),
             'val': torch.from_numpy(y_train).float().to(self.device)}
        
       #training set is used as the validation set in the anomaly detection task
#         X = {'train': torch.from_numpy(X_train).float().to(self.device),
#             'val': torch.from_numpy(X_test_val).float().to(self.device)}

#         y = {'train': torch.from_numpy(y_train).float().to(self.device),
#             'val': torch.from_numpy(y_test_val).float().to(self.device)}

        task_type = 'binclass'
        n_classes = None
        d_out = n_classes or 1

        if self.model_name == 'ResNet':
            self.model = rtdl.ResNet.make_baseline(
                d_in=X_train.shape[1],
                d_main=128,
                d_hidden=256,
                dropout_first=params['dropout_first'],
                dropout_second=0.0,
                n_blocks=params['n_blocks'],
                d_out=d_out,
            )
            lr = params['learning_rate']
            weight_decay = 0.0
        
        elif self.model_name == 'MLP':
            self.model = rtdl.MLP.make_baseline(
            d_in=X_train.shape[1],
            d_layers= [128, 256, 128],
            dropout=0.25,
            d_out=d_out,
            )
            lr = 0.001
            weight_decay = 0.0

        elif self.model_name == 'FTTransformer':
            self.model = rtdl.FTTransformer.make_default(
                n_num_features=X_train.shape[1],
                cat_cardinalities=None,
                last_layer_query_idx=[-1],  # it makes the model faster and does NOT affect its output
                d_out=d_out,
            )
            
        elif self.model_name == 'FTTransformer_baseline':
            self.model = rtdl.FTTransformer.make_baseline(
                n_num_features=X_train.shape[1],
                cat_cardinalities=None,
                d_token=X_train.shape[1],
                n_blocks=2,
                attention_dropout=0.2,
                ffn_d_hidden=6,
                ffn_dropout=0.2,
                residual_dropout=0.0,
                d_out=d_out,
            ) 
        else:
            raise NotImplementedError

        self.model.to(self.device)
        optimizer = (
            self.model.make_default_optimizer()
            if isinstance(self.model, rtdl.FTTransformer)
            else torch.optim.AdamW(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        )
        loss_fn = (
            F.binary_cross_entropy_with_logits
            if task_type == 'binclass'
            else F.cross_entropy
            if task_type == 'multiclass'
            else F.mse_loss
        )

        # Create a dataloader for batches of indices
        # Docs: https://yura52.github.io/zero/reference/api/zero.data.IndexLoader.html
        train_loader = delu.data.IndexLoader(len(X['train']), self.batch_size, device=self.device)

        # Create a progress tracker for early stopping
        # Docs: https://yura52.github.io/zero/reference/api/zero.ProgressTracker.html
        progress = delu.ProgressTracker(patience=100)

        # training
        # report_frequency = len(X['train']) // self.batch_size // 5
        aucroc = []
        aucpr = []
        loss_ = []
        for epoch in range(1, self.n_epochs + 1):
            loss_tmp = []
            for iteration, batch_idx in enumerate(train_loader):
                self.model.train()
                optimizer.zero_grad()
                x_batch = X['train'][batch_idx]
                y_batch = y['train'][batch_idx]
                loss = loss_fn(self.apply_model(x_batch).squeeze(1), y_batch)
                loss_tmp.append(loss.item())
                loss.backward()
                optimizer.step()
                # if iteration % report_frequency == 0:
                #     print(f'(epoch) {epoch} (batch) {iteration} (loss) {loss.item():.4f}')

            loss_.append(sum(loss_tmp)/len(loss_tmp))
            _, val_metric = self.evaluate(X=X['val'], y=y['val'])
            print(f'Epoch {epoch:03d} | Validation metric: {val_metric:.4f}', end='')
            progress.update((-1 if task_type == 'regression' else 1) * val_metric)
            if progress.success:
                print(' <<< BEST VALIDATION EPOCH', end='')
            print()
            # 验证
            # output predicted anomaly score on testing set
            score = self.predict_score(X_test)
            # evaluation
            result = utils.metric(y_true=y_test, y_score=score)
            aucroc.append(result['aucroc'])
            aucpr.append(result['aucpr'])
            if progress.fail:
                break
        return result['aucroc']
        #return self

    def predict_score(self, X):
        X = torch.from_numpy(X).float().to(self.device)
        score, _ = self.evaluate(X=X, y=None)
        return score

D:\autosoftware\anconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
data_path3 = "Test_A"
epoch = 10
batch_size = 256
model_name = "ResNet"
# 加载训练集的pkl文件，划分训练集与验证集
ind_pkl_files = []#存放标签为0的文件
ood_pkl_files = []#存放标签为1的文件
data_path="Train"#存放数据的路径
pkl_files = glob(data_path+'/*.pkl')
for each_path in tqdm(pkl_files):
    pic = open(each_path,'rb')
    this_pkl_file= pickle.load(pic)#下载pkl文件
    if this_pkl_file[1]['label'] == '00':
        ind_pkl_files.append(each_path)
    else:
        ood_pkl_files.append(each_path)

random.seed(0)
# 排序并打乱存放车辆序号的集合
random.shuffle(ind_pkl_files)
random.shuffle(ood_pkl_files)
# 3/4的正样本和全部的负样本作为训练集，1/4的正样本和1/4的负样本作为训练集
train_pkl_files = [ ind_pkl_files[j] for j in range(len(ind_pkl_files)//4,len(ind_pkl_files))] + [ ood_pkl_files[i] for i in range(len(ood_pkl_files))]
test_pkl_files=[ind_pkl_files[i] for i in range(len(ind_pkl_files)//4)] + [ood_pkl_files[i] for i in range(len(ood_pkl_files)//4)]

print(len(train_pkl_files))
print(len(test_pkl_files))

# 加载并归一化训练数据和验证数据
X_train,y_train=load_data(train_pkl_files)
# 进行随机打乱，这里random_state指定为固定值，则打乱结果相同
X_train,y_train = shuffle(X_train,y_train,random_state=40)
X_test,y_test=load_data(test_pkl_files)
X_train = normalization(X_train)
X_test = normalization(X_test)

test1_files = glob(data_path3+'/*.pkl')
X_val,_=load_data(test1_files,label=False)
X_val = normalization(X_val)

100%|███████████████████████████████████████████████████████████████████████████| 28389/28389 [02:25<00:00, 195.32it/s]


22456
7096


In [3]:
def objective(trial):
    seed = 42
    clf=FTTransformer(seed,model_name,n_epochs=10,batch_size=batch_size)

    params = {
              'learning_rate': trial.suggest_float("lr", 1e-5, 1e-1, log=True),
              'dropout_first': trial.suggest_float('dropout_first', 0.1, 0.5),
              'n_blocks': trial.suggest_int("n_blocks", 1, 4),
              }

    accuracy = clf.fit(X_train=X_train, y_train=y_train.squeeze(1),X_test=X_test,y_test=y_test,params=params)

    return accuracy

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=10)
best_trial = study.best_trial
print(best_trial.value)
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

[I 2022-10-15 13:47:28,315] A new study created in memory with name: no-name-81b9ca6f-4362-4647-9141-85a4106304e0


Epoch 001 | Validation metric: 0.8191 <<< BEST VALIDATION EPOCH
Epoch 002 | Validation metric: 0.8968 <<< BEST VALIDATION EPOCH
Epoch 003 | Validation metric: 0.9391 <<< BEST VALIDATION EPOCH
Epoch 004 | Validation metric: 0.9649 <<< BEST VALIDATION EPOCH
Epoch 005 | Validation metric: 0.9723 <<< BEST VALIDATION EPOCH
Epoch 006 | Validation metric: 0.9779 <<< BEST VALIDATION EPOCH
Epoch 007 | Validation metric: 0.9819 <<< BEST VALIDATION EPOCH
Epoch 008 | Validation metric: 0.9835 <<< BEST VALIDATION EPOCH
Epoch 009 | Validation metric: 0.9843 <<< BEST VALIDATION EPOCH


[I 2022-10-15 13:47:57,402] Trial 0 finished with value: 0.9961790292545926 and parameters: {'lr': 0.0002662521240223481, 'dropout_first': 0.1427002844927043, 'n_blocks': 2}. Best is trial 0 with value: 0.9961790292545926.


Epoch 010 | Validation metric: 0.9886 <<< BEST VALIDATION EPOCH
Epoch 001 | Validation metric: 0.8856 <<< BEST VALIDATION EPOCH
Epoch 002 | Validation metric: 0.9420 <<< BEST VALIDATION EPOCH
Epoch 003 | Validation metric: 0.9418
Epoch 004 | Validation metric: 0.9629 <<< BEST VALIDATION EPOCH
Epoch 005 | Validation metric: 0.9721 <<< BEST VALIDATION EPOCH
Epoch 006 | Validation metric: 0.9759 <<< BEST VALIDATION EPOCH
Epoch 007 | Validation metric: 0.9738
Epoch 008 | Validation metric: 0.9648
Epoch 009 | Validation metric: 0.9813 <<< BEST VALIDATION EPOCH
Epoch 010 | Validation metric: 0.9690


[I 2022-10-15 13:48:47,804] Trial 1 finished with value: 0.9899077967078348 and parameters: {'lr': 0.004846560836959535, 'dropout_first': 0.3464939660858418, 'n_blocks': 4}. Best is trial 0 with value: 0.9961790292545926.


...

Epoch 001 | Validation metric: 0.6309 <<< BEST VALIDATION EPOCH
Epoch 002 | Validation metric: 0.6842 <<< BEST VALIDATION EPOCH
Epoch 003 | Validation metric: 0.7171 <<< BEST VALIDATION EPOCH
Epoch 004 | Validation metric: 0.7386 <<< BEST VALIDATION EPOCH
Epoch 005 | Validation metric: 0.7555 <<< BEST VALIDATION EPOCH
Epoch 006 | Validation metric: 0.7715 <<< BEST VALIDATION EPOCH
Epoch 007 | Validation metric: 0.7832 <<< BEST VALIDATION EPOCH
Epoch 008 | Validation metric: 0.7943 <<< BEST VALIDATION EPOCH
Epoch 009 | Validation metric: 0.8051 <<< BEST VALIDATION EPOCH
Epoch 010 | Validation metric: 0.8162 <<< BEST VALIDATION EPOCH


[I 2022-10-15 13:52:56,763] Trial 8 finished with value: 0.9419960264223063 and parameters: {'lr': 1.670363720616462e-05, 'dropout_first': 0.28172697177222206, 'n_blocks': 3}. Best is trial 4 with value: 0.996462069492248.


Epoch 001 | Validation metric: 0.8770 <<< BEST VALIDATION EPOCH
Epoch 002 | Validation metric: 0.9346 <<< BEST VALIDATION EPOCH
Epoch 003 | Validation metric: 0.9519 <<< BEST VALIDATION EPOCH
Epoch 004 | Validation metric: 0.9507
Epoch 005 | Validation metric: 0.9727 <<< BEST VALIDATION EPOCH
Epoch 006 | Validation metric: 0.9780 <<< BEST VALIDATION EPOCH
Epoch 007 | Validation metric: 0.9714
Epoch 008 | Validation metric: 0.9827 <<< BEST VALIDATION EPOCH
Epoch 009 | Validation metric: 0.9888 <<< BEST VALIDATION EPOCH


[I 2022-10-15 13:53:16,078] Trial 9 finished with value: 0.9952678802662983 and parameters: {'lr': 0.01572347497043378, 'dropout_first': 0.13099749231550045, 'n_blocks': 1}. Best is trial 4 with value: 0.996462069492248.


Epoch 010 | Validation metric: 0.9845
0.996462069492248
lr: 0.000422874917555388
dropout_first: 0.26307843861624325
n_blocks: 3
